# Start coding and Enjoy the journey

## Install the repo

In [ ]:
!git clone --recursive https://github.com/Digre01/Visual-Place-Recognition-Project.git

## Install dependencies

In [ ]:
%cd Visual-Place-Recognition-Project/image-matching-models
!pip install -e .[all]

In [ ]:
!pip install faiss-cpu
!pip install loguru
!pip install yacs

## Download Datasets

In [ ]:
%cd Visual-Place-Recognition-Project

In [ ]:
!python download_datasets.py

## Run your First VPR Evalutation



In [ ]:
!python VPR-methods-evaluation/main.py \
--num_workers 4 \
--batch_size 32 \
--log_dir log_dir \
--method=netvlad --backbone=VGG16 --descriptors_dimension=4096 \
--image_size 512 512 \
--database_folder 'data/tokyo_xs/test/database' \
--queries_folder 'data/tokyo_xs/test/queries' \
--num_preds_to_save 20 \
--recall_values 1 5 10 20 \
--faiss_metric "L2" \
--save_for_uncertainty 


## Visualization Inliers from comparison Correct queries vs Wrong queries

In [ ]:
import os
from glob import glob
from pathlib import Path
from util import get_list_distances_from_preds 
import numpy as np
import torch
import matplotlib.pyplot as plt


PREDS_DIR = "path_to_predictions"
INLIERS_DIR = "path_to_predictions+'_matcher-name' "   

POSITIVE_DIST_THRESHOLD = 25.0
NUM_BINS = 50

def get_top1_distance_from_preds(txt_file):
    try:
        dists = get_list_distances_from_preds(txt_file)
        if dists and len(dists) > 0:
            return dists[0]
    except Exception as e:
        print(f"Error: could not read {txt_file}: {e}")
    return None


preds_folder = Path(PREDS_DIR)
inliers_folder = Path(INLIERS_DIR)

txt_files = glob(os.path.join(preds_folder, "*.txt"))
txt_files.sort(key=lambda x: int(Path(x).stem) if Path(x).stem.isdigit() else Path(x).stem)

correct_inliers = []
wrong_inliers = []
num_missing_inliers = 0

for txt_file in txt_files:
    query_id = Path(txt_file).stem

    top1_dist = get_top1_distance_from_preds(txt_file)
    if top1_dist is None:
        continue
    is_correct = top1_dist <= POSITIVE_DIST_THRESHOLD

    torch_file = inliers_folder / f"{query_id}.torch"

    query_results = torch.load(torch_file, weights_only=False)
    if not isinstance(query_results, (list, tuple)) or len(query_results) == 0:
        continue

    top1_result = query_results[0]
    num_inliers = int(top1_result.get("num_inliers", 0))

    if is_correct:
        correct_inliers.append(num_inliers)
    else:
        wrong_inliers.append(num_inliers)

correct_inliers = np.array(correct_inliers, dtype=np.int32)
wrong_inliers = np.array(wrong_inliers, dtype=np.int32)

print(f"Total queries with preds:            {len(txt_files)}")
print(f"Correct R@1 queries:                 {len(correct_inliers)}")
print(f"Wrong R@1 queries:                   {len(wrong_inliers)}")

print("\ Inliers statistics:")
print(f"  Correct queries inliers: mean={correct_inliers.mean():.1f}, "
      f"median={np.median(correct_inliers):.1f}, ")
print(f"  Wrong queries inliers: mean={wrong_inliers.mean():.1f}, "
      f"median={np.median(wrong_inliers):.1f}, ")

    # ============================================
    # Plot histograms
    # ============================================
plt.figure(figsize=(8, 5))

max_inliers = max(correct_inliers.max(), wrong_inliers.max())
bins = np.linspace(0, max_inliers, NUM_BINS + 1)

plt.hist(
    correct_inliers,
    bins=bins,
    alpha=0.6,
    label=f"Correct (N={len(correct_inliers)})",
    color="tab:green",
    edgecolor="black",
)
plt.hist(
    wrong_inliers,
    bins=bins,
    alpha=0.6,
    label=f"Wrong (N={len(wrong_inliers)})",
    color="tab:red",
    edgecolor="black",
)

plt.xlabel("Number of inliers for top‑1 retrieved image")
plt.ylabel("Number of queries")
plt.title(f"Inliers distribution (threshold = {POSITIVE_DIST_THRESHOLD} m)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Running Matching with Adaptive thresholding

In [ ]:
#TESTED ON 10 20 30 45 65 80


!python match_queries_preds.py \
--preds-dir 'path_to_predition' \
--matcher 'matcher-name' \
--device 'cuda' \
--num-preds 20 \
--adaptive-threshold 10

In [ ]:
!python adaptive_reranking.py  \
    --preds-dir 'path_to_predition' \
    --inliers-dir 'path_to_prediction+"_matcher-name"' \
    --num-preds 20

## Check Re-ranking Performance

In [ ]:
!python reranking.py \
--preds-dir '<path-to-predictions-folder>' \
--inliers-dir '<path-to-inliers-folder>' \
--num-preds 20 \
--recall-values 1 5 10 20

## Perform Uncertainty Evalutation

In [ ]:
!python -m vpr_uncertainty.eval \
--preds-dir '<path-to-predictions-folder>' \
--inliers-dir '<path-to-inliers-folder>' \
--z-data-path '<path-to-z-data-file>'